# Class Parser Workflow Test

This notebook reproduces the workflow from poc.ipynb step by step using the new src/ structure.
Test each component individually and then run the complete pipeline.

## Workflow Overview
1. **Data Extraction** - Extract slides and plan from PDF (deterministic)
2. **Branch A** - With plan: outline_two_pass → mapping_two_pass → enrich → writer
3. **Branch B** - No plan: outline_one_shot → enrich → writer
4. **Export** - Generate DOCX document


In [1]:
# Setup and imports
import os
import json
import logging
import sys
from pathlib import Path

# Configure logging for the src package
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set our package to DEBUG level for detailed progress
logging.getLogger("src").setLevel(logging.DEBUG)

# Load environment variables
import dotenv
dotenv.load_dotenv()

# Import our src modules
from src import (
    Course, CoursePipeline, Content, ContentSection, 
    MappingItem, SectionSlideMapping, Slides
)
from src.data_extraction import SlidesExtractor, PlanExtractor
from src.llm import (
    OutlineOneShot, OutlineTwoPass, MappingTwoPass, Writer, PromptManager
)
# Ensure src is importable when running the notebook directly
import sys
sys.path.append(str(Path('.').resolve()))

print("✅ All imports successful!")
print(f"📁 Working directory: {os.getcwd()}")

# Check if volume directory exists
volume_path = Path("./volume")
slides_path = volume_path / "slides"
print(f"📂 Volume directory exists: {volume_path.exists()}")
print(f"📂 Slides directory exists: {slides_path.exists()}")

if slides_path.exists():
    pdf_files = list(slides_path.glob("*.pdf"))
    print(f"📄 PDF files found: {len(pdf_files)}")
    for pdf in pdf_files:
        print(f"  - {pdf.name}")


✅ All imports successful!
📁 Working directory: /Users/youssefjanjar/Documents/formascience/class_parser
📂 Volume directory exists: True
📂 Slides directory exists: True
📄 PDF files found: 3
  - cours_4_plan.pdf
  - cours_1.pdf
  - cours_4.pdf


In [53]:
# Dev hot-reload setup (no kernel restart needed)
%load_ext autoreload
%autoreload 2

import sys, importlib
from pathlib import Path
# Ensure project root is on sys.path
sys.path.insert(0, str(Path('.').resolve()))

import src
import src.models as models
import src.llm as llm
import src.data_extraction as data_extraction
import src.course as course_mod
import src.pipeline as pipeline_mod


def refresh_imports():
    """Reload modules and rebind notebook aliases."""
    importlib.reload(src)
    importlib.reload(models)
    importlib.reload(llm)
    importlib.reload(data_extraction)
    importlib.reload(course_mod)
    importlib.reload(pipeline_mod)
    
    globals().update({
        # Models
        "Content": models.Content,
        "ContentSection": models.ContentSection,
        "MappingItem": models.MappingItem,
        "SectionSlideMapping": models.SectionSlideMapping,
        "Slides": models.Slides,
        "OutlineAndMapping": models.OutlineAndMapping,
        # LLM
        "OutlineOneShot": llm.OutlineOneShot,
        "OutlineTwoPass": llm.OutlineTwoPass,
        "MappingTwoPass": llm.MappingTwoPass,
        "Writer": llm.Writer,
        "PromptManager": llm.PromptManager,
        # Deterministic extractors
        "SlidesExtractor": data_extraction.SlidesExtractor,
        "PlanExtractor": data_extraction.PlanExtractor,
        # Orchestrators
        "Course": course_mod.Course,
        "CoursePipeline": pipeline_mod.CoursePipeline,
    })
    print("🔁 Imports refreshed")

# Initial bind
refresh_imports()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 Imports refreshed


## Step 1: Data Extraction - Slides

Extract slides from PDF using the exact logic from poc.ipynb


In [5]:
# Test slides extraction
pdf_path = "./volume/slides/cours_1.pdf"  # Update this path as needed

# Initialize extractor
slides_extractor = SlidesExtractor(
    min_avg_len=10,
    max_lines=20,
    merge_tol=2.0
)

print(f"🔍 Extracting slides from: {pdf_path}")

try:
    # Extract slides as Slides objects
    slides = slides_extractor.extract_slides(pdf_path)
    print(f"✅ Extracted {len(slides)} slides")
    
    # Display first few slides
    print("\n📋 First 3 slides:")
    for i, slide in enumerate(slides[:3]):
        print(f"\n--- Slide {i+1}: {slide.id} ---")
        print(f"Title: {slide.title}")
        print(f"Content (first 200 chars): {slide.content[:200]}...")
        print("=" * 50)
        
except FileNotFoundError:
    print(f"❌ PDF file not found: {pdf_path}")
    print("Please ensure the PDF file exists in ./volume/slides/")
    slides = []  # Empty list for testing
except Exception as e:
    print(f"❌ Error extracting slides: {e}")
    slides = []


🔍 Extracting slides from: ./volume/slides/cours_1.pdf
2025-08-17 16:02:26,950 [DEBUG] src.data_extraction.slides_extractor: Starting slide extraction from ./volume/slides/cours_1.pdf
2025-08-17 16:02:27,002 [DEBUG] src.data_extraction.slides_extractor: PDF opened, found 111 pages
2025-08-17 16:02:27,744 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P412' is an invalid float value
2025-08-17 16:02:27,746 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P414' is an invalid float value
2025-08-17 16:02:27,761 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P444' is an invalid float value
2025-08-17 16:02:27,822 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P451' is an invalid float value
2025-08-17 16:02:27,823 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P455' is an invalid float value
2025-08-17 16:02:27,838 [WARNING] pdfminer.pdfinterp: Cannot set gray non-

# Single Shot

# Single shot:  Extract the plan for this class 

In [6]:
outline_one_shot = OutlineOneShot()

outline, mapping = outline_one_shot.build_outline_and_mapping(slides[:20])

print(outline.print_outline())

2025-08-17 16:02:39,149 [DEBUG] src.llm.outline_one_shot: Starting one-shot outline and mapping generation with 20 slides
2025-08-17 16:04:34,809 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-17 16:04:34,867 [DEBUG] src.llm.outline_one_shot: Successfully generated outline with 5 sections and mapping with 13 entries
Course Outline:
Présentation du module et plan du cours
Introduction au contexte évolutif et organismes modèles
  Arbre de la vie et concept de LUCA
  Arbre des eucaryotes et choix d'organismes modèles
Notions fondamentales et définitions clés
  Homologie: définition, orthologie, paralogie et méthodes de détection
  Biologie du gène, dogme central et omiques
Le génome humain: composition et organisation générale
  Composition générale du génome: haploïde et diploïde
Détermination et séquençage du génome humain
  Historique, objectifs et ciblage de l’assemblage initial
  Cartographie génétique basée sur fréquences de recombinai

In [11]:
outline 

Content(sections=[ContentSection(id='SEC_1', title='Présentation du module et plan du cours', content=['L1SpS: UE 2 Les molécules du vivant\nU.E.2 Les molécules du vivant\n7. Organisation du génome humain, méthodes en\nbiotechnologie\n7.1 Architecture du génome\nhumain\nJean Muller\nLaboratoire Diagnostic Génétique (HUS)\nLaboratoire de Génétique médicale (Inserm U1112)\njeanmuller@unistra.fr', 'L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2'], subsections=[]), ContentSection(id='SEC_2', title='Introduction au contexte év

In [12]:
print(mapping.visualize_mapping(outline))

Section-to-Slides Mapping:

[Root] SEC_1
Title: Présentation du module et plan du cours
Slides: 2 slide(s)
Slide IDs: SL_001, SL_002

[Root] SEC_2
Title: Introduction au contexte évolutif et organismes modèles
Slides: 1 slide(s)
Slide IDs: SL_003

  [Level 1] SEC_2.1
  Title: Arbre de la vie et concept de LUCA
  Slides: 1 slide(s)
  Slide IDs: SL_004

  [Level 1] SEC_2.2
  Title: Arbre des eucaryotes et choix d'organismes modèles
  Slides: 1 slide(s)
  Slide IDs: SL_005

[Root] SEC_3
Title: Notions fondamentales et définitions clés
Slides: 1 slide(s)
Slide IDs: SL_006

  [Level 1] SEC_3.1
  Title: Homologie: définition, orthologie, paralogie et méthodes de détection
  Slides: 5 slide(s)
  Slide IDs: SL_006, SL_007, SL_008, SL_009, SL_010

  [Level 1] SEC_3.2
  Title: Biologie du gène, dogme central et omiques
  Slides: 2 slide(s)
  Slide IDs: SL_011, SL_012

[Root] SEC_4
Title: Le génome humain: composition et organisation générale
Slides: 1 slide(s)
Slide IDs: SL_013

  [Level 1] SEC_

# Single shot: Writing content from mapping and the outline

In [13]:
outline_with_slides = outline.enrich_with_slides(slides=slides, mapping=mapping)
writer = Writer()

course = writer.write_course(enriched_content=outline_with_slides)

type(course)

2025-08-17 16:07:26,856 [DEBUG] src.llm.writer: Starting content enhancement for 5 sections
2025-08-17 16:10:02,399 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-17 16:10:02,491 [DEBUG] src.llm.writer: Successfully enhanced content with 5 sections


src.models.Content

In [24]:
type(course)

src.models.Content

In [25]:
print(course.print_content())

Course Content:

[Root] ID: SEC_1
Title: Présentation du module et plan du cours
Content:
  [1] Ce module porte sur l'organisation du génome humain et sur les méthodes
      biotechnologiques permettant de le caractériser et de l'analyser. Il vise
      à donner une vision intégrée de la composition, de l'architecture et de la
      dynamique des génomes, ainsi que des conséquences fonctionnelles et
      médicales des variations génomiques.
  [2] Le plan du cours présente successivement une introduction aux concepts
      évolutifs et aux organismes modèles, les notions fondamentales de
      génétique moléculaire, la description du génome humain (nucléaire et
      mitochondrial), le projet de séquençage, l'architecture globale du génome,
      la description des éléments constitutifs (gènes codants et non codants,
      pseudogènes, éléments répétés), la comparaison intergénomique, les
      différents types de variations et leurs conséquences, la variabilité du
      génome humain 

# Store a course as a artifact that I can load again into Course

In [35]:
refresh_imports()
# Get the raw data
content_data = course.model_dump()

# Recreate with current Content class
from src.models import Content
fresh_content = Content.model_validate(content_data)

pilot_course = Course(
    name="Architecture du génome humain",
    course_title="Architecture du génome humain",
    level="L1",
    block="SANTE",
    semester="S1",
    subject="UE-1 Constitution et transformation de la matière",
    chapter="CHAPITRE_9",
    content=fresh_content
)

pilot_course

🔁 Imports refreshed


Course(name='Architecture du génome humain', course_title='Architecture du génome humain', level='L1', block='SANTE', semester='S1', subject='UE-1 Constitution et transformation de la matière', chapter='CHAPITRE_9', content=Content(sections=[ContentSection(id='SEC_1', title='Présentation du module et plan du cours', content=["Ce module porte sur l'organisation du génome humain et sur les méthodes biotechnologiques permettant de le caractériser et de l'analyser. Il vise à donner une vision intégrée de la composition, de l'architecture et de la dynamique des génomes, ainsi que des conséquences fonctionnelles et médicales des variations génomiques.", "Le plan du cours présente successivement une introduction aux concepts évolutifs et aux organismes modèles, les notions fondamentales de génétique moléculaire, la description du génome humain (nucléaire et mitochondrial), le projet de séquençage, l'architecture globale du génome, la description des éléments constitutifs (gènes codants et non

In [36]:
pilot_course.save_to_json()

💾 Course saved to: volume/artifacts/architecture_du_génome_humain_20250817_162748.json


'volume/artifacts/architecture_du_génome_humain_20250817_162748.json'

In [32]:
# Check what's actually in your fresh_content variable
print(f"Type: {type(fresh_content)}")
print(f"Is Content? {isinstance(fresh_content, Content)}")

# Check the sections
if hasattr(fresh_content, 'sections'):
    print(f"Sections type: {type(fresh_content.sections)}")
    if fresh_content.sections:
        print(f"First section type: {type(fresh_content.sections[0])}")
        print(f"First section: {fresh_content.sections[0]}")
else:
    print("No sections attribute")

# Check if it's the right Content class
print(f"Content class: {Content}")
print(f"fresh_content's content class: {type(fresh_content).__module__}.{type(fresh_content).__name__}")

Type: <class 'src.models.Content'>
Is Content? True
Sections type: <class 'list'>
First section type: <class 'src.models.ContentSection'>
First section: id='SEC_1' title='Présentation du module et plan du cours' content=["Ce module porte sur l'organisation du génome humain et sur les méthodes biotechnologiques permettant de le caractériser et de l'analyser. Il vise à donner une vision intégrée de la composition, de l'architecture et de la dynamique des génomes, ainsi que des conséquences fonctionnelles et médicales des variations génomiques.", "Le plan du cours présente successivement une introduction aux concepts évolutifs et aux organismes modèles, les notions fondamentales de génétique moléculaire, la description du génome humain (nucléaire et mitochondrial), le projet de séquençage, l'architecture globale du génome, la description des éléments constitutifs (gènes codants et non codants, pseudogènes, éléments répétés), la comparaison intergénomique, les différents types de variation

# Two-shots

# Two-shots

## Plan from page

Extract plan text from PDF for Branch A (two-pass approach)


## Extract the slides

In [70]:
pdf_path = './volume/slides/cours_1.pdf'
plan_path = './volume/slides/cours_4_plan.pdf'

extractor = SlidesExtractor()

slides = extractor.extract_slides(pdf_path=pdf_path)

slides[:5]



2025-08-18 10:12:25,875 [DEBUG] src.data_extraction.slides_extractor: Starting slide extraction from ./volume/slides/cours_1.pdf
2025-08-18 10:12:25,924 [DEBUG] src.data_extraction.slides_extractor: PDF opened, found 111 pages
2025-08-18 10:12:26,844 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P412' is an invalid float value
2025-08-18 10:12:26,845 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P414' is an invalid float value
2025-08-18 10:12:26,861 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P444' is an invalid float value
2025-08-18 10:12:26,876 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P451' is an invalid float value
2025-08-18 10:12:26,877 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P455' is an invalid float value
2025-08-18 10:12:26,894 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P465' is an invalid float value

[Slides(id='SL_001', title='7.1 Architecture du génome', content='L1SpS: UE 2 Les molécules du vivant\nU.E.2 Les molécules du vivant\n7. Organisation du génome humain, méthodes en\nbiotechnologie\n7.1 Architecture du génome\nhumain\nJean Muller\nLaboratoire Diagnostic Génétique (HUS)\nLaboratoire de Génétique médicale (Inserm U1112)\njeanmuller@unistra.fr'),
 Slides(id='SL_002', title='Plan du cours', content='L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2'),
 Slides(id='SL_003', title='Introduction', content='L1SpS: UE 2

In [74]:
# Test plan extraction
plan_extractor = PlanExtractor()

plan_txt = plan_extractor.extract_plan_from_page(pdf_path=pdf_path, page_number=2)

print(plan_txt)

2025-08-18 10:13:00,872 [DEBUG] src.data_extraction.plan_extractor: Starting plan extraction from ./volume/slides/cours_1.pdf (page 2)
2025-08-18 10:13:00,928 [DEBUG] src.data_extraction.plan_extractor: PDF opened, found 111 pages
2025-08-18 10:13:00,948 [DEBUG] src.data_extraction.plan_extractor: Successfully extracted plan from page 2 (1599 chars)
   L1SpS: UE 2 Les molécules du vivant                                                             
      Plan      du     cours                                                                       
        •  Introduction                                                                            
             •  Notions  fondamentales                                                             
        •  Le génome    humain                                                                     
             •  Projet de séquençage                                                               
             •  Architecture globale            

## Build the Outline

In [75]:
out_builder = OutlineTwoPass()

outline = out_builder.build_outline(plan_txt)

2025-08-18 10:13:26,958 [DEBUG] src.llm.outline_two_pass: Starting outline generation from plan text (1599 chars)
2025-08-18 10:13:41,765 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-18 10:13:41,817 [DEBUG] src.llm.outline_two_pass: Successfully generated outline with 6 sections


In [78]:
print(outline.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
  Description des principaux éléments constituants
    Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


## Build the mapping

In [79]:
slides = slides[:10]

In [80]:
mapping_builder = MappingTwoPass()

mapping = mapping_builder.build_mapping(slides=slides, outline=outline) 

type(mapping)

2025-08-18 10:14:02,625 [DEBUG] src.llm.mapping_two_pass: Starting mapping generation for 10 slides and 6 sections
2025-08-18 10:15:20,967 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-18 10:15:20,996 [DEBUG] src.llm.mapping_two_pass: Successfully generated mapping with 13 entries


src.models.SectionSlideMapping

In [82]:
print(mapping.visualize_mapping(outline))

Section-to-Slides Mapping:

[Root] SEC_1
Title: Introduction
Slides: 2 slide(s)
Slide IDs: SL_002, SL_003

  [Level 1] SEC_1.1
  Title: Notions fondamentales
  Slides: 7 slide(s)
  Slide IDs: SL_004, SL_005, SL_006, SL_007, SL_008, SL_009, SL_010

[Root] SEC_2
Title: Le génome humain
Slides: 1 slide(s)
Slide IDs: SL_001

  [Level 1] SEC_2.1
  Title: Projet de séquençage
  Slides: 1 slide(s)
  Slide IDs: SL_002

  [Level 1] SEC_2.2
  Title: Architecture globale
  Slides: 1 slide(s)
  Slide IDs: SL_001

    [Level 2] SEC_2.2.1
    Title: Génome mitochondrial
    Slides: 1 slide(s)
    Slide IDs: SL_001

    [Level 2] SEC_2.2.2
    Title: Génome nucléaire
    Slides: 1 slide(s)
    Slide IDs: SL_001

  [Level 1] SEC_2.3
  Title: Description des principaux éléments constituants
  Slides: 1 slide(s)
  Slide IDs: SL_001

    [Level 2] SEC_2.3.1
    Title: Gènes (codants et non-codants), pseudogènes et éléments répétés
    Slides: 1 slide(s)
    Slide IDs: SL_001

[Root] SEC_3
Title: Comparai

In [87]:
course_with_slides = outline.enrich_with_slides(slides=slides, mapping=mapping)

print(course_with_slides.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
  Description des principaux éléments constituants
    Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


## Write the course

In [84]:
writer = Writer()

course = writer.write_course(enriched_content=course_with_slides)
type(course)

2025-08-18 10:16:07,926 [DEBUG] src.llm.writer: Starting content enhancement for 6 sections
2025-08-18 10:19:58,868 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-18 10:19:58,980 [DEBUG] src.llm.writer: Successfully enhanced content with 6 sections


src.models.Content

In [89]:
type(course)

src.models.Content

In [88]:
test_course = Course(
    name="Architecture du génome humain",
    course_title="Architecture du génome humain",
    level="L1",
    block="SANTE",
    semester="S1",
    subject="UE-1 Constitution et transformation de la matière",
    content=course
)

print(test_course.print_outline())

ValidationError: 1 validation error for Course
content
  Input should be a valid dictionary or instance of Content [type=model_type, input_value=Content(sections=[Content...ides=None, mapping=None), input_type=Content]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [90]:
from src.models import Content as ModelsContent
print("Course content annotation ->", ModelsContent, ModelsContent.__module__)

print("Returned content instance ->", type(course), type(course).__module__)

print("Same class object? ->", type(course) is ModelsContent)

Course content annotation -> <class 'src.models.Content'> src.models
Returned content instance -> <class 'src.models.Content'> src.models
Same class object? -> False


## Tests


In [64]:
refresh_imports()

from src.models import PipelineConfig

config = PipelineConfig.load("pipeline.yaml")
config.inputs, config.outputs, config.metadata

🔁 Imports refreshed


({'slides_pdf': '/Users/youssefjanjar/Documents/formascience/class_parser/volume/slides/cours_1.pdf',
  'plan_page': 2},
 {'save_json': True,
  'save_docx': True,
  'template_path': '/Users/youssefjanjar/Documents/formascience/class_parser/volume/templates/fs_template.docx',
  'output_dir': '/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts'},
 CourseMetadata(name='Architecture du génome humain', course_title='Architecture du génome humain', level='L1', block='SANTE', semester='S1', subject='UE-1 - Les bases de la biologie', chapter=None, year=2024, professor='Youssef Janjar'))

In [65]:
import os 
os.listdir(config.outputs['output_dir'])

['architecture_du_génome_humain_20250817_224951.json',
 '.DS_Store',
 'architecture_du_génome_humain_filled.docx',
 'docx']

In [66]:
from src.course import Course

course = Course.load_from_json(config.outputs['output_dir']+'/architecture_du_génome_humain_20250817_224951.json')

print(course.content.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
  Description des principaux éléments constituants
    Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


In [67]:
course.save_to_json(output_path=config.outputs['output_dir'])

💾 Course saved to: /Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/architecture_du_génome_humain_20250818_082858.json


'/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/architecture_du_génome_humain_20250818_082858.json'

In [61]:
course.to_docx(template_path=config.outputs['template_path'], output_path=config.outputs['output_dir']+'/docx')

📄 Course exported to DOCX: /Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/docx/architecture_du_génome_humain.docx


'/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/docx/architecture_du_génome_humain.docx'

# Testing the pipeline

In [68]:
import os
from pathlib import Path

# Print current working directory
print("Current working directory:", os.getcwd())

# Print root folder using pathlib
root_path = Path.cwd()
print("Root folder (pathlib):", root_path)

# Print absolute path
print("Absolute path:", root_path.absolute())

# List contents of root folder
print("\nContents of root folder:")
for item in root_path.iterdir():
    if item.is_dir():
        print(f"📁 {item.name}/")
    else:
        print(f"📄 {item.name}")


Current working directory: /Users/youssefjanjar/Documents/formascience/class_parser
Root folder (pathlib): /Users/youssefjanjar/Documents/formascience/class_parser
Absolute path: /Users/youssefjanjar/Documents/formascience/class_parser

Contents of root folder:
📁 .cursor/
📄 .DS_Store
📄 create_structure.sh
📁 audio_extractor/
📄 pyproject.toml
📄 data_structure.md
📁 tests/
📁 __pycache__/
📄 README.md
📁 .mypy_cache/
📄 .gitignore
📄 diagram.mermaid
📄 .env
📁 .venv/
📄 pipeline.yaml
📄 poetry.lock
📁 .git/
📄 main.py
📄 test_workflow.ipynb
📁 data/
📁 volume/
📁 notebooks/
📁 src/


In [1]:
import logging
import sys
from pathlib import Path

from src.pipeline import CoursePipeline
from src.models import CourseMetadata

# Configure logging level
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG for more detailed logs, WARNING for less
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # Console output
        logging.FileHandler('pipeline.log')  # Optional: save to file
    ]
)

# Initialize the pipeline
pipeline = CoursePipeline()
# Method 1: Using config file
course = pipeline.process_from_config("configs/cours_4.yaml")


# Get processing statistics
stats = pipeline.get_processing_statistics(course)
print(f"Processed {stats['total_sections']} sections from {stats['total_slides']} slides")

ValidationError: 1 validation error for CourseMetadata
chapter
  Input should be a valid string [type=string_type, input_value=4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [9]:
import os

jsons = os.listdir("/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/")
file_path = "/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/" + jsons[-1]

file_path

'/Users/youssefjanjar/Documents/formascience/class_parser/volume/artifacts/json/architecture_du_génome_humain_20250818_114200.json'

In [10]:
from src.course import Course

course = Course.load_from_json(
file_path)
course

Course(name='Architecture du génome humain', course_title='Architecture du génome humain', level='L1', block='SANTE', semester='S1', subject='UE-1', chapter=None, content=Content(sections=[ContentSection(id='SEC_1', title='Introduction', content=["Ce cours propose une vue d'ensemble de la biologie des molécules du vivant, centrée sur la structure, la fonction, la variabilité et l'évolution du génome humain. Les thèmes abordés couvrent les notions fondamentales d'évolution et d'homologie, la description du génome humain (nucléaire et mitochondrial), le projet de séquençage, l'architecture globale du génome, les principaux constituants (gènes codants et non codants, pseudogènes, éléments répétés), la comparaison avec d'autres génomes, les types de variations génomiques et leurs conséquences, ainsi que les mécanismes évolutifs responsables des remaniements génomiques.", "L'objectif pédagogique est de fournir aux étudiants les concepts et le vocabulaire nécessaires pour comprendre comment 

In [11]:
course.subject

'UE-1'

In [7]:
course.save_to_json(output_path="volume/artifacts/json")

💾 Course saved to: volume/artifacts/json/architecture_du_génome_humain_20250818_114200.json


'volume/artifacts/json/architecture_du_génome_humain_20250818_114200.json'

In [12]:
atifacts_path = "volume/artifacts/docx"
template_path = "volume/templates/fs_template.docx"
course.to_docx(output_path=atifacts_path, template_path=template_path)

📄 Course exported to DOCX: volume/artifacts/docx/architecture_du_génome_humain.docx


'volume/artifacts/docx/architecture_du_génome_humain.docx'

## Step 1: Data Extraction - Slides

Extract slides from PDF using the exact logic from poc.ipynb


In [ ]:
# Method 2: Direct method call
metadata = CourseMetadata(
    name="Architecture du génome humain",
    course_title="Architecture du génome humain",
    level="L1",
    block="SANTE",
    semester="S1",
    subject="UE-1 Constitution et transformation de la matière"
)

course = pipeline.process_course_no_plan(
    slides=slides,  # Your slides list
    metadata=metadata,
    save_json=True,
    save_docx=True,
    test_mode=True,  # Process only first 10 slides
    output_path=""
)

# Get processing statistics
stats = pipeline.get_processing_statistics(course)
print(f"Processed {stats['total_sections']} sections from {stats['total_slides']} slides")